In [2]:
#!pip install mediapipe
#!pip install opencv-python

In [67]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
from IPython.display import Image
import pandas as pd

In [37]:
def find_connected_cameras():
    connected_cameras = []
    for i in range(10):
        cap = cv2.VideoCapture(i)
        if not cap.isOpened():
            break
        else:
            connected_cameras.append(i)
            cap.release()
    return connected_cameras
cameras = find_connected_cameras()
cameras

[0, 1]

In [69]:
![Image Description](https://i.imgur.com/3j8BPdc.png)
Image(url='https://i.imgur.com/3j8BPdc.png', height=300)

'[Image' is not recognized as an internal or external command,
operable program or batch file.


In [145]:
def angle_calculator(a,b,c):
    a = a.x,a.y
    b = b.x,b.y
    c = c.x,c.y
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180/np.pi)
    if angle > 180.0:
        angle = 360-angle
        
    return angle,b

In [271]:
cap = cv2.VideoCapture(0)
left_count = 0
center_count = 0
right_count = 0
left_stage = None
center_stage = None
right_stage = None
width = 1280
height = 720
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)


    while cap.isOpened():
        ret, frame = cap.read()
        cv2.imshow("Frame", frame)
        results = pose.process(frame)
        frame.flags.writeable = True
        
        try:
            landmarks = results.pose_landmarks.landmark
            left_angle, left_elbow = angle_calculator(landmarks[11],landmarks[13],landmarks[15])
            right_angle, right_elbow = angle_calculator(landmarks[12],landmarks[14],landmarks[16])
            #left_hip_angle, left_hip = angle_calculator(landmarks[11],landmarks[23],landmarks[25])
            #right_hip_angle, right_hip = angle_calculator(landmarks[12],landmarks[24],landmarks[26])            
            left_shoulder_angle, left_shoulder = angle_calculator(landmarks[13],landmarks[11],landmarks[23])
            right_shoulder_angle, right_shoulder = angle_calculator(landmarks[14],landmarks[12],landmarks[24])           
            cv2.putText(frame, str(left_angle),tuple(np.multiply(left_elbow, [width,height]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255, 0), 1, cv2.LINE_AA)
            cv2.putText(frame, str(right_angle),tuple(np.multiply(right_elbow, [width,height]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255, 0), 1, cv2.LINE_AA)
            cv2.putText(frame, str(left_shoulder_angle),tuple(np.multiply(left_shoulder, [width,height]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255, 0), 1, cv2.LINE_AA)
            cv2.putText(frame, str(right_shoulder_angle),tuple(np.multiply(right_shoulder, [width,height]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255, 0), 1, cv2.LINE_AA)
            #cv2.putText(frame, str(left_hip_angle),tuple(np.multiply(left_hip, [width,height]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255, 0), 1, cv2.LINE_AA)
            #cv2.putText(frame, str(right_hip_angle),tuple(np.multiply(right_hip, [width,height]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255, 0), 1, cv2.LINE_AA)
        
        
            if left_shoulder_angle < 40:
                if left_angle > 160:
                    left_stage = 'Down'
                if left_angle<30 and left_stage == 'Down':
                    left_stage="Up"
                    left_count += 1
            if right_shoulder_angle < 40:
                if right_angle > 160:
                    right_stage = 'Down'
                if right_angle<30 and right_stage == 'Down':
                    right_stage="Up"
                    right_count += 1
                
            if left_shoulder_angle < 30 and right_shoulder_angle < 30:
                center_stage='Down'
            if left_shoulder_angle > 160 and right_shoulder_angle > 160 and center_stage=='Down':
                center_stage='Up'
                center_count += 1
            
            cv2.rectangle(frame,(0,0),(230,85),(255,255,255),-1)
            cv2.putText(frame,'Left Curl',(75,14), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1, cv2.LINE_AA)
            cv2.putText(frame,'Reps',(15,42), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 1, cv2.LINE_AA)
            cv2.putText(frame, str(left_count),(15,77), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 1, cv2.LINE_AA)                                
            cv2.putText(frame,'Position',(100,42), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 1, cv2.LINE_AA)
            cv2.putText(frame, left_stage,(100,77), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 1, cv2.LINE_AA)        
        
            cv2.rectangle(frame,(1050,0),(1280,85),(255,255,255),-1)
            cv2.putText(frame,'Right Curl',(1125,14), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1, cv2.LINE_AA)
            cv2.putText(frame,'Reps',(1065,42), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 1, cv2.LINE_AA)
            cv2.putText(frame, str(right_count),(1065,77), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 1, cv2.LINE_AA)                                
            cv2.putText(frame,'Position',(1150,42), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 1, cv2.LINE_AA)
            cv2.putText(frame, right_stage,(1150,77), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 1, cv2.LINE_AA)
            
            cv2.rectangle(frame,(525,0),(755,85),(255,255,255),-1)
            cv2.putText(frame,'Center',(600,14), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1, cv2.LINE_AA)
            cv2.putText(frame,'Reps',(540,42), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 1, cv2.LINE_AA)
            cv2.putText(frame, str(center_count),(540,77), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 1, cv2.LINE_AA)                                
            cv2.putText(frame,'Position',(625,42), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 1, cv2.LINE_AA)
            cv2.putText(frame, center_stage,(625,77), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 1, cv2.LINE_AA)
            
            
        except:
            pass
        
        
        mp_drawing.draw_landmarks(frame,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(255,255,255),thickness=1,circle_radius=1),
                                  mp_drawing.DrawingSpec(color=(255,255,255),thickness=1,circle_radius=1)
                                 )
        cv2.imshow("Frame", frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
cap.release()
cv2.destroyAllWindows()

In [269]:
len(landmarks)

33

In [133]:
llist = []
for i in mp_pose.PoseLandmark:
    i = str(i)
    llist.append(i.split('.')[1])

### NOW FOR LEFT HAND LET'S FIND ANGLE AND IT IS BETWEEN Landmark (11,13,15)

In [139]:
print(llist[11],landmarks[11])
print(llist[13],landmarks[13])
print(llist[15],landmarks[15])

LEFT_SHOULDER x: 0.7296022176742554
y: 0.8091387152671814
z: -0.08228478580713272
visibility: 0.994053304195404

LEFT_ELBOW x: 0.7729374170303345
y: 1.1786272525787354
z: -0.2805488407611847
visibility: 0.4545603096485138

LEFT_WRIST x: 0.7438985109329224
y: 1.0512274503707886
z: -0.8717184066772461
visibility: 0.8629314303398132



In [147]:
a,b = angle_calculator(landmarks[11],landmarks[13],landmarks[15])

In [151]:
tuple(np.multiply(b,[width,height]).astype(int))

(1159, 848)

### NOW FOR RIGHT HAND LET'S FIND ANGLE AND IT IS BETWEEN Landmark (12,14,16)

In [140]:
print(llist[12],landmarks[12])
print(llist[14],landmarks[14])
print(llist[16],landmarks[16])

RIGHT_SHOULDER x: 0.4533967971801758
y: 0.7605755925178528
z: -0.5645679235458374
visibility: 0.9970878958702087

RIGHT_ELBOW x: 0.3735935688018799
y: 1.1479250192642212
z: -0.8871262669563293
visibility: 0.6700240969657898

RIGHT_WRIST x: 0.46911782026290894
y: 1.2072503566741943
z: -1.2502150535583496
visibility: 0.7237216234207153



In [144]:
angle_calculator(landmarks[12],landmarks[14],landmarks[16])

110.20093644285647